# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab Notebooks/fifa-world-cup-2018/source/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Read files

In [ ]:
import pandas as pd
pd.set_option("max_columns", 100)

In [ ]:
players_origin = pd.read_json(open(path + "players.json", encoding="utf8"))
teams_origin = pd.read_json(open(path + "teams.json", encoding="utf8"))
matches_origin = pd.read_json(open(path + "matches_World_Cup.json", encoding="utf8"))
events_origin = pd.read_json(open(path + "events_World_Cup.json", encoding="utf8"))

# Convert nested data in matches

In [ ]:
matchesInfo = pd.DataFrame(
  columns=[
    "matchId",
    "gameweek",
    "homeAway",
    "teamId",
    "score",
    "scoreP"
  ]
)

matchesMember = pd.DataFrame(
  columns=[
    "matchId",
    "teamId",
    "homeAway",
    "playerId",
    "startingF",
    "goals",
    "ownGoals",
    "yellowCards",
    "redCards"
  ]
)

In [ ]:
for matchId, gameweek, teams in zip(matches_origin["wyId"], matches_origin["gameweek"], matches_origin["teamsData"]):
  for team, info in teams.items():
    if str(info["side"]) == "home":
      homeAway = 1
    else:
      homeAway = 2
    matchesInfo = matchesInfo.append(
        pd.Series(
            [
              matchId,
              gameweek,
              homeAway,
              team,
              info["score"],
              info["scoreP"]
            ],
            index=matchesInfo.columns
        ),
        ignore_index=True
    )

    for startingM in info["formation"]["lineup"]:
      matchesMember = matchesMember.append(
          pd.Series(
              [
                  matchId,team,
                  homeAway,
                  startingM["playerId"],
                  1,
                  startingM["goals"].replace("null","0"),
                  startingM["ownGoals"],
                  startingM["yellowCards"],
                  startingM["redCards"]
              ],
              index=matchesMember.columns
          ),
          ignore_index=True
      )
    for benchM in info["formation"]["bench"]:
      matchesMember = matchesMember.append(
        pd.Series(
            [
              matchId
              ,team
              ,homeAway
              ,benchM["playerId"]
              ,0
              ,benchM["goals"].replace("null","0")
              ,benchM["ownGoals"]
              ,benchM["yellowCards"]
              ,benchM["redCards"]
            ],
            index=matchesMember.columns
        ),
        ignore_index=True
      )

In [ ]:
matchesInfo.head()

,matchId,gameweek,homeAway,teamId,score,scoreP
0,2058017,0,2,9598,2,0
1,2058017,0,1,4418,4,0
2,2058016,0,2,2413,0,0
3,2058016,0,1,5629,2,0
4,2058015,0,2,2413,1,0


In [ ]:
matchesMember.head()

,matchId,teamId,homeAway,playerId,startingF,goals,ownGoals,yellowCards,redCards
0,2058017,9598,2,69616,1,0,0,0,0
1,2058017,9598,2,105361,1,0,0,0,0
2,2058017,9598,2,69409,1,0,0,92,0
3,2058017,9598,2,25393,1,0,0,0,0
4,2058017,9598,2,135747,1,0,0,0,0


# Convert dict data in teams and players

In [ ]:
from pandas.io.json import json_normalize

teams = pd.DataFrame(
  data={
    "teamId": teams_origin["wyId"]
    ,"name": teams_origin["name"]
    ,"officialName": teams_origin["officialName"]
  }
).join(
  pd.DataFrame(
    data={
      "areaCode": json_normalize(teams_origin["area"])["alpha3code"]
      ,"areaName": json_normalize(teams_origin["area"])["name"]
    }
  )
)
players = pd.DataFrame(
  data={
    "playerId": players_origin["wyId"]
    ,"clubTeamId": players_origin["currentTeamId"]
    ,"nationalTeamTd":players_origin["currentNationalTeamId"].replace("null","0")
    ,"playerName": players_origin["shortName"]
    ,"firstName": players_origin["firstName"]
    ,"middleName": players_origin["middleName"]
    ,"lastName": players_origin["lastName"]
  }
).join(
  pd.DataFrame(
    data={
      "positionCode": json_normalize(players_origin["role"])["code2"]
      ,"positionName": json_normalize(players_origin["role"])["name"]
    }
  )
)

<ipython-input-8-3a1a73b6f78f>:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  "areaCode": json_normalize(teams_origin["area"])["alpha3code"]
<ipython-input-8-3a1a73b6f78f>:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  ,"areaName": json_normalize(teams_origin["area"])["name"]
<ipython-input-8-3a1a73b6f78f>:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  "positionCode": json_normalize(players_origin["role"])["code2"]
<ipython-input-8-3a1a73b6f78f>:31: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  ,"positionName": json_normalize(players_origin["role"])["name"]


In [ ]:
teams

,teamId,name,officialName,areaCode,areaName
0,1613,Newcastle United,Newcastle United FC,XEN,England
1,692,Celta de Vigo,Real Club Celta de Vigo,ESP,Spain
2,691,Espanyol,Reial Club Deportiu Espanyol,ESP,Spain
3,696,Deportivo Alav\u00e9s,Deportivo Alav\u00e9s,ESP,Spain
4,695,Levante,Levante UD,ESP,Spain
...,...,...,...,...,...
137,1598,Spain,Spain,ESP,Spain
138,17322,Serbia,Serbia,SRB,Serbia
139,6697,Switzerland,Switzerland,CHE,Switzerland
140,13869,Poland,Poland,POL,Poland


In [ ]:
players

,playerId,clubTeamId,nationalTeamTd,playerName,firstName,middleName,lastName,positionCode,positionName
0,32777,4502,4687,H. Tekin,Harun,,Tekin,GK,Goalkeeper
1,393228,3775,4423,M. Sarr,Malang,,Sarr,DF,Defender
2,393230,3772,0,O. Mandanda,Over,,Mandanda,GK,Goalkeeper
3,32793,683,19314,A. N'Diaye,Alfred John Momar,,N'Diaye,MD,Midfielder
4,393247,2975,0,I. Konat\u00e9,Ibrahima,,Konat\u00e9,DF,Defender
...,...,...,...,...,...,...,...,...,...
3598,120839,16041,0,A. Ma\u00e2loul,Ali,,Ma\u00e2loul,DF,Defender
3599,114736,15591,0,C. C\u00e1ceda,Carlos Alberto,,C\u00e1ceda Oyaguez,GK,Goalkeeper
3600,114908,12072,0,M. Araujo,Miguel Gianpierre,,Araujo Blanco,DF,Defender
3601,285583,16183,0,A. Tagnaouti,Ahmed Reda,,Tagnaouti,GK,Goalkeeper


# Convert events

tagId:
* [101 (Goal)](https://dataglossary.wyscout.com/shot/)
* [302 (Key pass)](https://dataglossary.wyscout.com/key_pass/)
* [1801 (Successful pass)](https://dataglossary.wyscout.com/pass/)

In [ ]:
events = pd.DataFrame(
  columns=[
    "eventId",
    "matchId",
    "matchPeriod",
    "teamId",
    "playerId",
    "beforeEventSec",
    "eventSec",
    "eventId",
    "subEventId",
    "goalF",
    "keyPass",
    "accurateF",
    "fromX",
    "fromY",
    "toX",
    "toY"
  ]
)

In [ ]:
beforeSec = 0
for index_event,event in events_origin.iterrows():
  #initialize variable
  fromX = -1
  fromY = -1
  toX = -1
  toY = -1
  goal_f = 0
  keypass_f = 0
  accurate_f = 0
  
  for pos in event["positions"]:
    if (fromX == -1 or fromY == -1):
      fromX = pos["x"]
      fromY = pos["y"]
    else:
      toX = pos["x"]
      toY = pos["y"]
  
  for tag in event["tags"]:
    if int(tag["id"]) == 101:
      goal_f = 1
    elif int(tag["id"]) == 302:
      keypass_f = 1
    elif int(tag["id"]) == 1801:
      accurate_f = 1
  
  events = events.append(
    pd.Series(
      [
        event["id"],
        event["matchId"],
        event["matchPeriod"],
        event["teamId"],
        event["playerId"],
        beforeSec,
        event["eventSec"],
        event["eventId"],
        event["subEventId"],
        goal_f,
        keypass_f,
        accurate_f,
        fromX,
        fromY,
        toX,
        toY
      ],
      index=events.columns
    ),
    ignore_index=True
  )
  
  beforeSec = event["eventSec"]

In [ ]:
eventKinds = events_origin[["eventId", "eventName"]][~events_origin[["eventId", "eventName"]].duplicated()]
subEventKinds = events_origin[["subEventId", "subEventName"]][~events_origin[["subEventId", "subEventName"]].duplicated()]

In [ ]:
events

,eventId,matchId,matchPeriod,teamId,playerId,beforeEventSec,eventSec,eventId,subEventId,goalF,keyPass,accurateF,fromX,fromY,toX,toY
0,258612104,2057954,1H,16521,122671,0,1.656214,8,85,0,0,1,50,50,35,53
1,258612106,2057954,1H,16521,139393,1.656214,4.487814,8,83,0,0,1,35,53,75,19
2,258612077,2057954,1H,14358,103668,4.487814,5.937411,1,10,0,0,1,25,81,37,83
3,258612112,2057954,1H,16521,122940,5.937411,6.406961,1,10,0,0,0,75,19,63,17
4,258612110,2057954,1H,16521,122847,6.406961,8.562167,8,85,0,0,1,63,17,71,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101754,263885652,2058017,2H,9598,3476,2972.985039,2978.301867,8,85,0,0,1,46,20,64,6
101755,263885653,2058017,2H,9598,14812,2978.301867,2979.084611,7,72,0,0,0,64,6,82,2
101756,263885654,2058017,2H,9598,14812,2979.084611,2983.448628,8,80,0,0,0,82,2,100,100
101757,263885613,2058017,2H,4418,25381,2983.448628,2985.869275,4,40,0,0,0,0,0,18,98


# Export to csv

In [ ]:
matchesInfo.to_csv(path + "matches.csv", index=False)
matchesMember.to_csv(path + "matches_member.csv", index=False)
events.to_csv(path + "events.csv", index=False)
eventKinds.to_csv(path + "event_kinds.csv", index=False)
subEventKinds.to_csv(path + "sub_event_kinds.csv", index=False)
players.to_csv(path + "players.csv", index=False)
teams.to_csv(path + "teams.csv", index=False)